In [5]:
%pip install kagglehub

Note: you may need to restart the kernel to use updated packages.


## 1. Fetching data:

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("varpit94/apple-stock-data-updated-till-22jun2021")

print("Path to dataset files:", path)

/opt/anaconda3/envs/dataenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 205k/205k [00:00<00:00, 7.76MB/s]

Extracting files...
Path to dataset files: /Users/minhnguyen/.cache/kagglehub/datasets/varpit94/apple-stock-data-updated-till-22jun2021/versions/8


In [6]:
!mv "/Users/minhnguyen/.cache/kagglehub/datasets/varpit94/apple-stock-data-updated-till-22jun2021/versions/8" "C:\Users\hweinguyen\Final Project\Architecture\AAPL.csv"

In [14]:
import pandas as pd

df = pd.read_csv('AAPL.csv')

df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1980-12-12,0.128348,0.128906,0.128348,0.128348,0.100323,469033600
1,1980-12-15,0.122210,0.122210,0.121652,0.121652,0.095089,175884800
2,1980-12-16,0.113281,0.113281,0.112723,0.112723,0.088110,105728000
3,1980-12-17,0.115513,0.116071,0.115513,0.115513,0.090291,86441600
4,1980-12-18,0.118862,0.119420,0.118862,0.118862,0.092908,73449600


## 2. Clean data:

In [15]:
df['Date'] = pd.to_datetime(df['Date'])
filter_year_df = df[(df['Date'].dt.year >= 2012 ) & (df['Date'].dt.year <= 2022)]
filter_year_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
7835,2012-01-03,14.621429,14.732143,14.607143,14.686786,12.575917,302220800
7836,2012-01-04,14.642857,14.810000,14.617143,14.765714,12.643503,260022000
7837,2012-01-05,14.819643,14.948214,14.738214,14.929643,12.783874,271269600
7838,2012-01-06,14.991786,15.098214,14.972143,15.085714,12.917513,318292800
7839,2012-01-09,15.196429,15.276786,15.048214,15.061786,12.897022,394024400


In [21]:
filter_year_df.isnull().any()

Date         False
Open         False
High         False
Low          False
Close        False
Adj Close    False
Volume       False
dtype: bool

In [22]:
filter_year_df[filter_year_df.duplicated()]

,Date,Open,High,Low,Close,Adj Close,Volume


In [23]:
cleaned_df = filter_year_df.sort_values(by='Date', ascending=True).reset_index(drop=True)
cleaned_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2012-01-03,14.621429,14.732143,14.607143,14.686786,12.575917,302220800
1,2012-01-04,14.642857,14.810000,14.617143,14.765714,12.643503,260022000
2,2012-01-05,14.819643,14.948214,14.738214,14.929643,12.783874,271269600
3,2012-01-06,14.991786,15.098214,14.972143,15.085714,12.917513,318292800
4,2012-01-09,15.196429,15.276786,15.048214,15.061786,12.897022,394024400


In [37]:
cleaned_df.to_csv('Cleaned_Apple_Data.csv')

## 3. Put the raw data and cleaned data into HDFS for further use:

In [26]:
%pip install hdfs

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for hdfs: filename=hdfs-2.7.3-py3-none-any.whl size=34325 sha256=d2de63ad7a80f9233288b68d1f02e23e321e12a79dedc54d4439268a2513db76
  Stored in directory: /Users/minhnguyen/Library/Caches/pip/wheels/97/ae/d9/536505928dd3a458b206013b02625df8f12d22fa154f2bfd65
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=543d6cd882bec049309e589a9eda21ead7b056595084c0f5aa7b3637a87d0e14
  Stored in directory: /Users/minhnguyen/Library/Caches/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built hdfs docopt
Note: you may need to restart the kernel to use updated packages.


In [28]:
!hdfs version

Hadoop 3.4.0
Source code repository git@github.com:apache/hadoop.git -r bd8b77f398f626bb7791783192ee7a5dfaeec760
Compiled by root on 2024-03-04T06:35Z
Compiled on platform linux-x86_64
Compiled with protoc 3.21.12
From source with checksum f7fe694a3613358b38812ae9c31114e
This command was run using /Users/minhnguyen/hadoop-3.4.0/share/hadoop/common/hadoop-common-3.4.0.jar


In [29]:
!jps

69219 SecondaryNameNode
69429 ResourceManager
68951 NameNode
69641 NodeManager
79711 Jps
69070 DataNode


### a. Data to HDFS:

In [36]:
!hdfs dfs -ls /user/minhnguyen/data 

2024-12-07 17:29:28,136 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
ls: `/user/minhnguyen/data': No such file or directory


In [44]:
from hdfs import InsecureClient

# Connect to the HDFS 
client = InsecureClient('http://localhost:9870', user='minhnguyen')

# Define the local raw_file cleaned_file path 
raw_data_path = '/Users/minhnguyen/Workspace/FINALPROJECT/ARCHITECTURE/AAPL.csv'
cleaned_data_path = '/Users/minhnguyen/Workspace/FINALPROJECT/ARCHITECTURE/Cleaned_Apple_Data.csv'

# Define the HDFS raw_file, cleaned_file path 
raw_hdfs_path = '/apple_stock_data/raw_data/Apple_Stock_1980_2022.csv'
cleaned_hdfs_path = '/apple_stock_data/cleaned_data/Cleaned_Apple_Stock.csv'

# Upload the file to HDFS
client.upload(raw_hdfs_path, raw_data_path)
client.upload(cleaned_hdfs_path, cleaned_data_path)

# List files in the HDFS directory
files_in_hdfs = client.list('/apple_stock_data')
print("Files in HDFS directory:", files_in_hdfs)

Files in HDFS directory: ['cleaned_data', 'raw_data']


## 4. Preprocess with PySpark

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('AppleStockAnomalyDetection').getOrCreate()


In [5]:
spark

In [13]:
df_pyspark = spark.read.format("csv").option("header", "true").load("Cleaned_Apple_Data.csv")
df_pyspark.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Adj Close: string (nullable = true)
 |-- Volume: string (nullable = true)



In [15]:
from pyspark.sql.functions import col 
from pyspark.sql.functions import to_date

df_pyspark = df_pyspark.select(
  to_date(col("Date"), "yyyy-MM-dd").alias("Date"),
  col("Open").cast("float"),
  col("High").cast("float"),
  col("Low").cast("float"),
  col("Close").cast("float"),
  col("Adj Close").cast("float"),
  col("Volume").cast("int"),
)

df_pyspark = df_pyspark.na.drop()

df_pyspark.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: float (nullable = true)
 |-- High: float (nullable = true)
 |-- Low: float (nullable = true)
 |-- Close: float (nullable = true)
 |-- Adj Close: float (nullable = true)
 |-- Volume: integer (nullable = true)



In [16]:
df_pyspark.head(5)

[Row(Date=datetime.date(2012, 1, 3), Open=14.621429443359375, High=14.73214340209961, Low=14.60714340209961, Close=14.686785697937012, Adj Close=12.57591724395752, Volume=302220800),
 Row(Date=datetime.date(2012, 1, 4), Open=14.64285659790039, High=14.8100004196167, Low=14.617142677307129, Close=14.765713691711426, Adj Close=12.643503189086914, Volume=260022000),
 Row(Date=datetime.date(2012, 1, 5), Open=14.819643020629883, High=14.948213577270508, Low=14.738213539123535, Close=14.929642677307129, Adj Close=12.783873558044434, Volume=271269600),
 Row(Date=datetime.date(2012, 1, 6), Open=14.991786003112793, High=15.098214149475098, Low=14.972143173217773, Close=15.085714340209961, Adj Close=12.917512893676758, Volume=318292800),
 Row(Date=datetime.date(2012, 1, 9), Open=15.196429252624512, High=15.276785850524902, Low=15.048213958740234, Close=15.061785697937012, Adj Close=12.897022247314453, Volume=394024400)]

## 5. Statistical Anomaly Detection: (Z-Score Method)

In [18]:
from pyspark.sql.functions import mean, stddev, abs

stats = df_pyspark.select(mean(col("Close")).alias("mean"), stddev(col("Close")).alias("stddev")).collect()
mean_close = stats[0]["mean"]
stddev_close = stats[0]["stddev"]

df_pyspark = df_pyspark.withColumn("Z-Score", (col("Close") - mean_close) / stddev_close)

anomalies = df_pyspark.filter(abs(col("Z-Score")) > 3)
anomalies.show()

+----------+------+------+------+------+---------+---------+------------------+
|      Date|  Open|  High|   Low| Close|Adj Close|   Volume|           Z-Score|
+----------+------+------+------+------+---------+---------+------------------+
|2021-12-10|175.21|179.63|174.69|179.45|179.22166|115402700| 3.035868074034137|
|2021-12-15|175.11| 179.5|172.31| 179.3|179.07185|131063300| 3.032304537672211|
|2021-12-27|177.09|180.42|177.07|180.33|180.10054| 74919600|3.0567751207345415|
|2021-12-28|180.16|181.33|178.53|179.29|179.06186| 79144300| 3.032066726903806|
|2021-12-29|179.33|180.63|178.14|179.38|179.15175| 62348900|3.0342052112373765|
|2021-12-30|179.47|180.57|178.09| 178.2|177.97325| 59773000|3.0061707292967053|
|2022-01-03|177.83|182.88|177.71|182.01| 181.7784|104487900|3.0966881780537707|
|2022-01-04|182.63|182.94|179.12| 179.7|179.47134| 99310400| 3.041807542981623|
+----------+------+------+------+------+---------+---------+------------------+



In [37]:
anomalies.write.option("header", True).option("delimiter", ",").mode("overwrite").csv("anomalies")

## 6. Put the analysis result into Amazon S3:

In [32]:
%pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 2.9 MB/s eta 0:00:00m eta 0:00:010:01:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os

aws_access_key = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")

In [2]:
import boto3

s3 = boto3.client(
  service_name = 's3',
  region_name = 'ca-central-1',
  aws_access_key_id = aws_access_key,
  aws_secret_access_key = aws_secret_key,
)

In [5]:
# store anomalies to S3
bucket_name = "apple-stock-analysis"
s3_key = "analysis-report/anomalies.csv"
file_name = "./anomalies/part-00000-9ca57c60-6132-4558-ba0c-596a32e4e038-c000.csv"

s3.upload_file(file_name, bucket_name, s3_key)

In [4]:
# store cleaned data to S3
bucket_name = "apple-stock-analysis"
s3_cleaned_key = "analysis-report/Cleaned_Apple_Data.csv"
cleaned_data_file = "./Cleaned_Apple_Data.csv"

s3.upload_file(cleaned_data_file, bucket_name, s3_cleaned_key)